In [18]:
import pandas as pd
from IPython.display import display
from IPython.display import Markdown as md

PRODUCTS_CSV = '../data/products_2018_08_30.csv'
ADQS_CSV = '../data/adqs_2017_06_01.csv'
SQUS_CSV = '../data/squs_2018_08_08.csv'
PROVIDED_ADQS_CSV = '../data/provided_adqs_2018_08_30.csv'


# Data sources

We depend on the following data sources to generate our data set:

*  `{ADQS_CSV}` (from an FOI request)
*  `{SQUS_CSV}` (generated by fetcher)
*  `{PRODUCTS_CSV}` (generated by SQL run against our postgres database)

We also use this source to validate our data set:

*  `{PROVIDED_ADQS_CSV}`: (generated by us from ADQs provided in NHS Digital dump)


## ADQs

For some indications, there is a wide variety of different products. For example, anxiety can be treated with a large number of different "benzos", which work in similar ways but have different molecules and therefore different strengths.  There are also different routes - benzos come as liquids, tablets and suppositories.

To compare prescribing for benzos across all different products, we use the concept of a Average Daily Quantity. It represents the assumed average maintenance dose per day for a drug used for its main indication in adults. The ADQ is not a recommended dose but an analytical unit to compare prescribing activity.

As far as we know, this is only available in ePACT (though back in 2012 it was available [as a PDF](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/9376/Average-daily-quantity-ADQ-values-2012-13/pdf/adqs_2012_13.pdf/)), and we have to request extracts over FOI ([example](https://apps.nhsbsa.nhs.uk/FOI/foiRequestDetail.do?bo_id=6938)).

The units are not standardised (there are litres and millilitres in there), and often the ADQ Unit field is blank, which 96% of the time we've worked out means "unit").


In [2]:
adqs = pd.read_csv(ADQS_CSV)
adqs.head()

,BNF Code,BNF Name,ADQ Value,ADQ Unit
0,010101000BBABA0,Langdales_Cinnamon Tab,NaN,NaN
1,0101010C0AAAAAA,Alum Hydrox_Cap 475mg,NaN,NaN
2,0101010C0AAACAC,Alum Hydrox_Oral Susp S/F,30.0,ML
3,0101010C0AAAHAH,Alum Hydrox_Tab 500mg,NaN,NaN
4,0101010C0BBAAAA,Alu-Cap_Cap 475mg,NaN,NaN


## SQUs

To apply ADQs to our prescribing data, we need to know what the `quantity` field means.  This is known as the **S**tandard **Q**uantity **U**nit (millilitres, grams, units, or "individually formulated").

We get this from the NHSBSA Prescription Cost Analysis dataset, which appears to be the only place the data is released (or mentioned) by the NHS.  It's available in monthly files relating to dispenser reimbursements, so we have written a scraper (`pca_quantity_fetcher.py`) which iterates over the spreadsheet for each month, looking for new BNF codes for which it doesn't have a mapping to SQU.  The result is `squs.csv`:

In [3]:
squs = pd.read_csv(SQUS_CSV)
squs.head()

,bnf_code,squ
0,0502050B0BEAACT,ml
1,0208010K0AABFBF,unit
2,091000000BBWAA0,unit
3,1502010J0BDABBR,unit
4,0703011A0BBAAAA,unit


In [4]:
squs[squs.bnf_code == '0411000G0BBABAB']

,bnf_code,squ
2126,0411000G0BBABAB,g
13631,0411000G0BBABAB,ml


## DMD Products

This CSV contains *form* information (e.g. `form_units` might be `ml` and `form_size` might be `5`) for VMPs.  

It also contains information about each of that VMP's ingredients (from the dm+d  `vpi` table).

These are all linked to BNF code. As some products have more than one ingredient, this means a BNF code can appear more than once.

Normally, knowing the unit of measure for the product is sufficient to map it to ADQs. However, sometimes the ADQ is denominated in a different unit of measure (e.g. a suspension where the unit of measure is a millilitre, but the ADQ is measured in grammes). The information about strength per millilitre (in this example) is only available at an ingredient level.

Of course, this fails to work with products containing more than one ingredient, in which case we currently fall back to parsing the BNF name (which usually encodes this information).

The product data is generated by a [SQL query](https://gist.github.com/sebbacon/c495b58803f85ce70dd26a311db7b432) against EBM DataLab's own Postgres representation of the dm+d, which is released weekly in an XML format by [NHS Digital](https://isd.digital.nhs.uk).  It is theoretically possible to reproduce this using [our code](https://github.com/ebmdatalab/openprescribing/tree/master/openprescribing/dmd), but this isn't currently easy for third parties.  



In [5]:
dmd = pd.read_csv(PRODUCTS_CSV)
dmd.head()

,bnf_code,name,vpid,form,form_indicator,form_size,form_units,unit_of_measure,numerator,numerator_unit_of_measure,denominator,denominator_unit_of_measure,ingredient_count
0,010101000BBAJA0,Indigestion mixture,11772211000001100,Oral suspension,Discrete,5.0,ml,spoonful,35.0,mg,1.0,ml,3.0
1,010101000BBAJA0,Indigestion mixture,11772211000001100,Oral suspension,Discrete,5.0,ml,spoonful,45.0,mg,1.0,ml,3.0
2,0101010A0BBAAAA,Actal 360mg tablets,4859911000001108,Tablet,Discrete,1.0,tablet,tablet,360.0,mg,NaN,NaN,1.0
3,0101010C0AAAAAA,Aluminium hydroxide 475mg capsules,316937008,Capsule,Discrete,1.0,capsule,capsule,475.0,mg,NaN,NaN,1.0
4,0101010C0AAAHAH,Aluminium hydroxide 500mg chewable tablets,316933007,Chewable tablet,Discrete,1.0,tablet,tablet,500.0,mg,NaN,NaN,1.0


In [6]:
indicator_vals = ", ".join(dmd.form_indicator.unique())
units_vals = ", ".join(dmd.form_units.fillna("").unique())
md("""
Some interesting fields and their possible values:

* `form_indicator`: {}
* `form_units`: {}
""".format(indicator_vals, units_vals))


Some interesting fields and their possible values:

* `form_indicator`: Discrete, Continuous, Not applicable
* `form_units`: ml, tablet, capsule, , sachet, plaster, kit, vial, suppository, application, actuation, pastille, enema, ampoule, patch, dose, pre-filled disposable injection, lozenge, film, cigarette, unit dose, bag, cartridge, piece, straw, microgram, device, gram, pessary, tube, applicator, system, litre, insert, dressing, strip, bottle, mg, pillule


## Provided ADQs

NHS Digital release a monthly [practices prescribing information dataset](https://apps.nhsbsa.nhs.uk/infosystems/data/showDataSelector.do?reportId=124), which includes a column that indicates ADQ Usage.  We already import the dataset monthly via BigQuery, so can use it to extract ADQs. We have no idea how this is calculated, but presuming it's correct, we can use it as a useful black box for validating our approach.

The file is generated by running the command `generate_data.py`.


In [7]:
provided_adqs = pd.read_csv(PROVIDED_ADQS_CSV)
provided_adqs.head()

,Unnamed: 0,Practice_Code,BNF_Code,BNF_Description,Items,Quantity,ADQ_Usage
0,0,P82016,0102000ACAAARAR,Atrop Sulf_Liq Spec 600mcg/5ml,1,200,20.00000
1,1,F85007,0102000P0BCABAB,Colofac_Tab 135mg,4,100,133.33332
2,2,E87045,0102000P0BHAAAB,Boots_IBS Relief Tab 135mg,1,120,39.99999
3,3,E87002,0102000T0AAAAAA,Peppermint Oil_Cap E/C 0.2ml,7,84,195.99998
4,4,M86006,0103010H0AAAAAA,Famotidine_Tab 20mg,1,28,14.00000


## Calculate ADQs

With all the above sources, we can now compute an ADQ per `quantity`.  Per the comment below, the logic is contained in the `adq_per_quantity` function in `adq_lib.py`.  You can read tests for it in `test_adq_lib.py`.


In [8]:
from adq import normalised_dmd
from adq import normalised_adqs
from adq import compute_quantity_units
from adq import adq_per_quantity

squs = pd.read_csv(SQUS_CSV)
# Add SQU info to the products table (normalised so all units are the same, 
# e.g. litres converted to millilitres)
df = normalised_dmd(PRODUCTS_CSV).merge(squs, on='bnf_code')

# Add a normalised version of the ADQs spreadsheet as well
df = df.merge(normalised_adqs(ADQS_CSV), on='bnf_code')

# Heuristically infer an SQU for anything where it wasn't available in the data.
df['quantity_denominator'] = df.apply(compute_quantity_units, axis=1)

# Compute an ADQ per quantity. See the `adq_per_quantity` function in `adq_lib` for details.
df['adq_per_quantity'] = df.apply(adq_per_quantity, axis=1)
df.to_csv("computed_adqs.csv")


In [9]:
interesting_columns = ['bnf_code', 'name', 'form_size', 'unit_of_measure', 'adq_value', 'adq_denominator', 'quantity_denominator', 'adq_per_quantity']
df2 = df[interesting_columns].groupby('bnf_code').first().reset_index()
df2.head()

,bnf_code,name,form_size,unit_of_measure,adq_value,adq_denominator,quantity_denominator,adq_per_quantity
0,0101010J0AAAAAA,Boots Magnesium trisilicate mixture,5.0,spoonful,30.0,ml,ml,0.033333
1,0101010L0BBABAA,Asilone oral suspension,5.0,spoonful,30.0,,ml,0.033333
2,0101010L0BBAEAK,Asilone Antacid liquid,5.0,spoonful,30.0,,ml,0.033333
3,0101010L0BEAAAI,Maalox Plus oral suspension,5.0,spoonful,30.0,,ml,0.033333
4,0101010Q0AAAGAG,Magnesium hydroxide 7.45-8.35% oral suspension BP,5.0,spoonful,30.0,ml,ml,0.033333


## Compare our ADQs against those provided by NHS Digital

In [10]:
import numpy as np
# Do QA check against NHS Digital data
comp = provided_adqs.merge(df2, left_on="BNF_Code", right_on="bnf_code")
comp.rename(columns={'ADQ_Usage': 'their_adq_usage'}, inplace=True)
comp['our_adq_usage'] = comp.apply(lambda x: (x.Quantity * x.Items * x.adq_per_quantity), axis=1)
delta = comp[comp['their_adq_usage'].round(1) != comp['our_adq_usage'].round(1)]
high = delta[delta['their_adq_usage'] > 500]
delta[interesting_columns].head()



,bnf_code,name,form_size,unit_of_measure,adq_value,adq_denominator,quantity_denominator,adq_per_quantity
18,0106020L0AAACAC,Glycerol 4g suppositories,1.0,suppository,4.00000,g,unit,0.7
147,0407020B0BHAAAH,Butec 5micrograms/hour transdermal patches,1.0,patch,0.00028,g,unit,NaN
148,0407020B0BKAAAH,Reletrans 5micrograms/hour transdermal patches,1.0,patch,0.00028,g,unit,NaN
149,0407020B0BLAAAH,Panitaz 5micrograms/hour transdermal patches,1.0,patch,0.00028,g,unit,NaN
150,0407020B0BNAAAE,Buplast 35micrograms/hour transdermal patches,1.0,patch,0.00112,g,unit,NaN


In [19]:
display(md("There are {} out of a total of {} ADQs that are possibly incorrect ({}%).  I estimate around a third of these are where we've calculated it right, and they've calculated it wrong.".format(len(delta), len(comp), round((len(delta)/float(len(comp))) * 100))))

display(md("{} where we were unable to compute an ADQ. {} of these are patches, which we can't make sense of anyway.".format(len(delta[np.isnan(delta.adq_per_quantity)]), len(delta[delta.unit_of_measure == 'patch']))))

display(md("{} are unit doses, where our algorithm needs fixing (there are three failing tests)".format(len(delta[(delta.unit_of_measure == 'unit dose') | (delta.unit_of_measure == 'dose')]))))

display(md("{} are forms of liquid ibuprofen whose ADQ was wrongly specified in millilitres when it should have been milligrams.".format(len(delta[delta.bnf_code.str.contains('1001010J0....BH')]))))

display(md("{} where their adq_usage is improbably high, so perhaps they're wrong.  {} are sachets, where I am pretty sure they've got it wrong. There are a few others where spot checking suggests their calculation is wrong - perhaps 10 more.".format(len(high), len(delta[delta.unit_of_measure == 'sachet']))))


There are 98 out of a total of 3276 ADQs that are possibly incorrect (3%).  I estimate around a third of these are where we've calculated it right, and they've calculated it wrong.

36 where we were unable to compute an ADQ. 35 of these are patches, which we can't make sense of anyway.

12 are unit doses, where our algorithm needs fixing (there are three failing tests)

8 are forms of liquid ibuprofen whose ADQ was wrongly specified in millilitres when it should have been milligrams.

4 where their adq_usage is improbably high, so perhaps they're wrong.  8 are sachets, where I am pretty sure they've got it wrong. There are a few others where spot checking suggests their calculation is wrong - perhaps 10 more.